## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Импортируем библиотеку pandas для работы с таблицами:

In [1]:
import pandas as pd

Запишем файл с входными данными в переменную data используя методы библиотеки pandas:

In [2]:
data = pd.read_csv('/datasets/data.csv')

Выведем первые 10 записей таблицы:

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Просмотрим сводную информацию о загруженной таблице: 

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Таблица содержит следующие сведения:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Всего в таблице 21525 строк, в столбцах days_employed и total_income количество записей меньше общего количества строк, это значит, что в данных столбцах имеются пропущенные значения.

В рамках поставленной задачи очень важны столбцы children, family_status и debt - именно в них содержится информация с помощью которой можно проверить интересующие заказчика зависимости.

### Шаг 2. Предобработка данных

### Обработка пропусков

Сначала убедимся в том, что названия столбцов таблицы не содержат лишних пробелов:

In [5]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Названия столбцов в порядке.

Теперь посчитаем количество пропусков в столбцах:

In [6]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения находятся в столбцах days_employed и total_income, оба столбца содержат количественные данные, следовательно для заполнения пропусков можно использовать характерные значения из выборки.

Начнем со столбца days_employed, вычислим среднее арифметическое:

In [7]:
data['days_employed'].mean()

63046.49766147338

Средний трудовой стаж составляет 172 года, цифра связана с тем что в исходных данных присутствует большое количество артефактов:

In [8]:
data.sort_values(by='days_employed', ascending=False).head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба


Среднее арифметическое для заполнения пропусков не годится, рассчитаем медиану:

In [9]:
data['days_employed'].median()

-1203.369528770489

Значение получилось отрицательным (-3 года), предположим это связано с тем что отрицательными значениями в исходном наборе данных отмечены периоды когда у заемщика не было постоянной работы в момент обращения за кредитом, в рамках проводимого исследования занятость работников не повлияет на выводы, заменим пропуски в столбце days_employed на медианное значение:

In [10]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())

Следующий столбец с пропущеными данными (total_income) содержит в себе информацию о ежемесячном доходе клиентов банка. Заполнять пропущенные значения средним арифметическим либо медианой в целом, без учета вида деятельности заемщика было бы не верным, разделим массив ежемесячного дохода по группам занятостей и рассчитаем медиану для каждого из них.

Создадим группировочную таблицу с медианными значениями по категориям занятости:

In [11]:
medians = data.groupby(['income_type']).agg({'total_income' : 'median'})
medians

,total_income
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,150447.935283
компаньон,172357.950966
пенсионер,118514.486412
предприниматель,499163.144947
сотрудник,142594.396847
студент,98201.625314


Заполним пропуски в столбце total_income для каждой категории занятости собственным значением:

In [12]:
for element in medians.index:
    data.loc[data['income_type'] == element, 'total_income'] = data.loc[data['income_type'] == element, 'total_income'].fillna(medians.loc[element, 'total_income'])

Убедимся в том, что в таблице не осталось пропущеных значений:

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

В исходном файле пропущенные значения обнаружены в столбцах days_employed и total_income, явных причин почему данные пропущены в таблице не представлено, мы можем предположить что пропуски связаны с:
1. Ошибками при заполнении базы данных хранящейся в банке
2. Ошибками при выгрузке информации

Кроме пропусков в столбце days_employed было обнаружено большое количество артефактов (их присутсвие может быть вызвано тем, что используются единицы измерения отличные от дней) которые не позволили воспользоваться средним арифметическим для заполнения пропусков в этом столбце.

Пропущеные значения в столбце total_income были заменены на медианные значене в разрезе вида занятости заемщика из столбца income_type.

### Замена типа данных

Выясним типы данных представленные в таблице:

In [14]:
data.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

Столбец days_employed содержит информацию об общем трудовом стаже в днях. Хранить информацию о днях в ячейках со значением типа информации float64 избыточно, произведем замену типа данных в данном столбце на int64:

In [15]:
data['days_employed'] = data['days_employed'].astype('int64')

Дробная часть сведений о доходе нам так же не понадобится, изменим тип данных в этом столбце на целочисленный: 

In [16]:
data['total_income'] = data['total_income'].astype('int64')

### Вывод

Мы изменили тип данных значений содержащихся в столбцах days_employed и total_income, в основном для удобства восприятия информации.

В связи с тем, что они изначально были представлены с типом float64 то был применен метод astype() с аргументом 'int64'.

### Обработка дубликатов

Проверим количество полностью совпадающих строк в таблице:

In [17]:
data.duplicated().sum()

54

Удалим полностью совпадающие строки методом drop_duplicates():

In [18]:
data = data.drop_duplicates().reset_index(drop=True)

Проверим столбцы содержащие категориальные данные на уникальность значений (education, family_status, gender, income_type, debt, purpose).

**education:**

In [19]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Категории образования отличаются регистром - переведем все значения в нижний:

In [20]:
data['education'] = data['education'].str.lower()

Проверим на уникальность:

In [21]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Повторяющихся значений не осталось.

**family_status:**

In [22]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Повторяющихся значений нет.

**gender:**

In [23]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Проверим сколько строк со значением xna в столбце gender:

In [24]:
data['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

Строка всего одна, выведем ее на экран:

In [25]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10690,0,-2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Кроме странного значения в поле gender остальная информация об этом заемщике в порядке, строку оставим, но если в исследовании понадобится провести анализ с использованием информации о поле заемщика - необходимо будет эту строку из анализа исключить. 

**income_type:**

In [26]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Повторяющихся значений нет.

In [27]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

**debt:**

In [28]:
data['debt'].unique()

array([0, 1])

Повторяющихся значений нет.

**purpose:**

In [29]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Значения в поле purpose ручному поиску дубликатов не поддадутся, на следующем шаге обработки данных необходимо будет произвести лемматизацию для разделения их на категории.

Проверим количественные значения в столбце **children** на артефакты:

In [30]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Среди адекватных значений имеются строки с количеством детей "-1" и "20", проверим их количество:

In [31]:
data['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Количество артефактов не большое, предположительно вызвано ошибками ввода данных, для исправления ситуации заменим "-1" на "1" а "20" на "2":

In [32]:
data['children'] = data['children'].replace({-1 : 1, 20 : 2})

Проверим значения:

In [33]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Содержимое столбца приведено к его названию.

Так как мы изменили регистр значений в поле education и заменили неадекватные значения в столбце children проверим таблицу на дубли еще раз:

In [34]:
data.duplicated().sum()

17

Удалим найденные дубликаты:

In [35]:
data = data.drop_duplicates().reset_index(drop=True)

Проверка:

In [36]:
data.duplicated().sum()

0

Дубли удалены.

### Вывод

Мы провели ручной поиск дубликатов в столбцах содержащих категориальные данные и обнаружили что в столбце с информацией об уровене образования клиента содержалось большое количество значений одинаковых по смыслу но разных по написанию.

Проверка значений столбца children на артефакты выявила два значения которые не соответствовали наименованию столбца.

После ручной замены неверных значений мы удалили дубли из таблицы и готовы приступить к лемматизации сведений о цели получения кредита.

### Лемматизация

Лемматизацию мы будем производить в столбце содержащем сведения о цели кредита. Так как цель кредитиа заемщики заявляют самостоятельно это неизбежно приводит к обозначению одних и тех же целей разными словосочетаниями:

In [37]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Подключим лемматизатор pymystem3 для слов на русском языке и коллекцию Counter для подсчета лемм:

In [38]:
from pymystem3 import Mystem
from collections import Counter

Для того чтобы провести лемматизацию уникальных значений столбца purpose соберем их в строку:

In [39]:
text_for_lemm = ''
for element in data['purpose'].unique():
    text_for_lemm = text_for_lemm + ', ' + element
text_for_lemm

', покупка жилья, приобретение автомобиля, дополнительное образование, сыграть свадьбу, операции с жильем, образование, на проведение свадьбы, покупка жилья для семьи, покупка недвижимости, покупка коммерческой недвижимости, покупка жилой недвижимости, строительство собственной недвижимости, недвижимость, строительство недвижимости, на покупку подержанного автомобиля, на покупку своего автомобиля, операции с коммерческой недвижимостью, строительство жилой недвижимости, жилье, операции со своей недвижимостью, автомобили, заняться образованием, сделка с подержанным автомобилем, получение образования, автомобиль, свадьба, получение дополнительного образования, покупка своего жилья, операции с недвижимостью, получение высшего образования, свой автомобиль, сделка с автомобилем, профильное образование, высшее образование, покупка жилья для сдачи, на покупку автомобиля, ремонт жилью, заняться высшим образованием'

Проведем лемматизацию и подсчет уникальных лемм среди целей приобретения кредита:

In [40]:
Counter(Mystem().lemmatize(text_for_lemm)).most_common()

[(' ', 59),
 (', ', 38),
 ('покупка', 10),
 ('недвижимость', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('с', 5),
 ('операция', 4),
 ('на', 4),
 ('свой', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('высокий', 3),
 ('дополнительный', 2),
 ('для', 2),
 ('коммерческий', 2),
 ('жилой', 2),
 ('подержать', 2),
 ('заниматься', 2),
 ('сделка', 2),
 ('приобретение', 1),
 ('сыграть', 1),
 ('проведение', 1),
 ('семья', 1),
 ('собственный', 1),
 ('со', 1),
 ('профильный', 1),
 ('сдача', 1),
 ('ремонт', 1),
 ('\n', 1)]

После лемматизации среди всех целей кредита можно выделить следующие действительные цели кредитов:

**недвижимость, автомобиль, образование, жилье, свадьба.**

Для того чтобы сопоставить цели кредита которые указали пользователи с действительными целями, полученными после лематизации, произведем следующие действия:

1. Создаем новый объект DataFrame для хранения информации о цели кредита:

In [41]:
purposes = pd.DataFrame(data = data['purpose'], columns=['purpose', 'lemmas', 'real_purpose'])

2. Удаляем дубликаты целей:

In [42]:
purposes = purposes.drop_duplicates().reset_index(drop=True)

3. Создаем функцию для лемматизации каждой цели кредита:

In [43]:
def lemmatization(text):
    lemmas = Mystem().lemmatize(text)
    return lemmas

4. Применяем эту функцию к целям хранящимся в созданном массиве:

In [44]:
purposes['lemmas'] = purposes['purpose'].apply(lemmatization)

5. Создаем функцию которая будет сопоставлять леммы целей кредита с одной из действительных целей кредитов (недвижимость, автомобиль, образование, жилье, свадьба) и возвращать действительную цель кредита которую мы в дальнейшем сможем использовать при анализе данных:

In [45]:
def real_purpose_find(text):
#в связи с тем что цель кредита "жилье" приоритетнее чем цель кредита "недвижимость"
#сначала проверим все что может указывать на то что кредит берется для приобретения жилья
    if 'жилье' in text:
        return 'жилье'
    elif 'жилой' in text:
        return 'жилье'
#и если приобретаемая недвижимость не является жилой отнесем ее к цели "недвижимость"
    elif 'недвижимость' in text:
        return 'недвижимость'
    elif 'автомобиль' in text:
        return 'автомобиль'
    elif 'образование' in text:
        return 'образование'
    elif 'свадьба' in text:
        return 'свадьба'

6. Применим эту функцию к леммам найденным в целях кредитов:

In [46]:
purposes['real_purpose'] = purposes['lemmas'].apply(real_purpose_find)

7. Выведем на экран первые 10 строк полученного массива данных:

In [47]:
purposes.head(10)

,purpose,lemmas,real_purpose
0,покупка жилья,"[покупка, , жилье, \n]",жилье
1,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,дополнительное образование,"[дополнительный, , образование, \n]",образование
3,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
4,операции с жильем,"[операция, , с, , жилье, \n]",жилье
5,образование,"[образование, \n]",образование
6,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба
7,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",жилье
8,покупка недвижимости,"[покупка, , недвижимость, \n]",недвижимость
9,покупка коммерческой недвижимости,"[покупка, , коммерческий, , недвижимость, \n]",недвижимость


8. Присоединим полученный массив данных к основной таблице методом merge по столбцу purpose:

In [48]:
data = data.merge(purposes, on='purpose', how='left')

Выведем на экран первые 10 строк новой таблицы:

In [49]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,real_purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",жилье
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",жилье
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",жилье
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",жилье
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",жилье


Убедимся в том что все цели кредитов заполнены:

In [50]:
data['real_purpose'].value_counts()

жилье           5690
недвижимость    5121
автомобиль      4306
образование     4013
свадьба         2324
Name: real_purpose, dtype: int64

### Вывод

Лемматизация данных вызвала некоторые затруднения в связи с ресурсоемкостью задачи. 
Задача была разделена на этапы:
* объединение всех целей кредита в одну и поиск наиболее часто встречающихся слов в ней. Это позволило нам сформировать перечень действительных целей кредитов;
* цели кредита из основной таблицы были скопированы в отдельный DataFrame в котором были удалены повторяющиеся значения, произведена лемматизация уникальных целей и был осуществлен поиск действительных целей кредитов среди найденных лемм;
* настоящие цели кредита были присоеденены к основной таблице.

Таким образом лемматизация была проведена с наименьшей нагрузкой на вычислительные мощности и для каждой строки основной таблицы мы определили действительную цель кредита которую сможем в дальнейшем использовать при анализе данных.

### Категоризация данных

При анализе данных нам понадобится разделять заемщиков по уровню ежемесячного дохода. Выделим следующие категории:
* менее 10 тысяч (ниже прожиточного минимума) - "<10";
* от 10 до 30 тысяч (ниже среднего дохода) - "10-30";
* от 30 до 50 тысяч (средний доход) - "30-50";
* от 50 до 100 тысяч (выше среднего дохода) - "50-100";
* от 100 до 150 тысяч (средний класс по уровню дохода) - "100-150";
* от 150 до 200 тысяч (верхняя граница среднего класса) - "150-200";
* более 200 тысяч - ">200".

Создадим функцию для определения категории дохода:

In [51]:
def income_category(income):
    if income <= 10000:
        return '<10'
    elif 10000 < income <= 30000:
        return '10-30'
    elif 30000 < income <= 50000:
        return '30-50'
    elif 50000 < income <= 100000:
        return '50-100'
    elif 100000 < income <= 150000:
        return '100-150'
    elif 150000 < income <= 200000:
        return '150-200'
    elif income > 200000:
        return '>200'  

Применим ее к нашей таблице и запишем результат в новый столбец income_category:

In [52]:
data['income_category'] = data['total_income'].apply(income_category)

Выведем на экран распеределение замещиков по уровню дохода:

In [53]:
data['income_category'].value_counts()

100-150    7160
>200       5067
150-200    4764
50-100     4091
30-50       350
10-30        22
Name: income_category, dtype: int64

Для анализа нам понадобится разделать заемщиков по наличию детей, определим функцию заменяющую количество детей в столбце на отметку о том есть дети или нет:

In [54]:
def children_replace(sum):
    if sum > 0:
        return 'Есть'
    else:
        return 'Нет'

Применим эту функцию и запишем результаты в новый столбец children_global:

In [55]:
data['children_global'] = data['children'].apply(children_replace)

Проверим результат работы функции:

In [56]:
data['children_global'].value_counts()

Нет     14091
Есть     7363
Name: children_global, dtype: int64

Выведем на экран первые 10 строк таблицы:

In [57]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,real_purpose,income_category,children_global
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",жилье,>200,Есть
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,100-150,Есть
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",жилье,100-150,Нет
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,>200,Есть
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,150-200,Нет
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",жилье,>200,Нет
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",жилье,>200,Нет
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование,100-150,Нет
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,50-100,Есть
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",жилье,100-150,Нет


### Вывод

Мы провели категорицию исходной информации по следующим столбцам:
* total_income — ежемесячный доход
* children — количество детей в семье

И добавили следующие столбцы к таблице:

* income_category - категория дохода заемщика в зависимости от суммы
* children_global - отметка о наличии детей

Информацию из новых столбцов (добавленных на этом этапе и столбца содержащего действительную цель кредита real_purpose) мы будем использовать на финальном этапе исследования.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Построим сводную таблицу для наглядного отображения количества заемщиков с детьи и без и количества задолженностей по кредиту:

In [58]:
children_global_pivot = data.pivot_table(index=['children_global'], values='debt', aggfunc=['count','sum'])
children_global_pivot['%'] = children_global_pivot['sum'] / children_global_pivot['count'] * 100
children_global_pivot.sort_values('%')

,count,sum,%
,debt,debt,
children_global,,,
Нет,14091,1063,7.543822
Есть,7363,678,9.208203


Зависимость между наличием детей и возвратом кредита в срок очень слабая, но присутствует. Среди заемщиков у которых нет детей доля имеющих задолженность по кредиту меньше на 1,66%. Добавим в сводную таблицу уровень дохода заемщика:

In [60]:
children_global_pivot = data.pivot_table(index=['income_category', 'children_global'], values='debt', aggfunc=['count','sum'])
children_global_pivot['%'] = children_global_pivot['sum'] / children_global_pivot['count'] * 100
children_global_pivot

count  sum          %
                                 debt debt           
income_category children_global                      
10-30           Есть                5    0   0.000000
                Нет                17    2  11.764706
100-150         Есть             2469  247  10.004050
                Нет              4691  377   8.036666
150-200         Есть             1673  161   9.623431
                Нет              3091  244   7.893885
30-50           Есть              107    9   8.411215
                Нет               243   12   4.938272
50-100          Есть             1300  124   9.538462
                Нет              2791  207   7.416697
>200            Есть             1809  137   7.573245
                Нет              3258  221   6.783303

Информацией о заемщиках с уровнем дохода от 10 до 30 тысяч можно пренебречь в связи с малым их количеством в исходной выборке (22)

Если принять в расчет доход заемщиков то с ростом его уровня влияние наличия детей на количество  замещиков у которых  есть задолженность по кредиту снижается:
	
* 30-50	3.47%
	
* 50-100 2.12%
	
* 100-150 1.97%
	
* 150-200 1.73%
	
* более 200 0.79%


### Вывод

Зависимость между наличием детей и возвратом кредита в срок присутствует и усугубляется со снижением уровня дохода заемщика.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Аналогично предыдущей гипотезе построим сводную таблицу для отображения информации:

In [61]:
family_status_pivot = data.pivot_table(index=['family_status'], values='debt', aggfunc=['count','sum'])
family_status_pivot['%'] = family_status_pivot['sum'] / family_status_pivot['count'] * 100
family_status_pivot.sort_values('%')

,count,sum,%
,debt,debt,
family_status,,,
вдовец / вдова,959,63,6.569343
в разводе,1195,85,7.112971
женат / замужем,12339,931,7.545182
гражданский брак,4151,388,9.347145
Не женат / не замужем,2810,274,9.750890


У состоящих в законном браке заемщиков чаще всего есть созаемщики в виде супругов, в связи с этим и количество задолженностей по кредиту среди этой категории ниже на 2,21% чем у не состоящих в браке и на 1,80% ниже чем у не оформивших свои отношения заемщиков.

Одновременно с этим, у вдовствующих и разведенных заемщиков уровень количества задолженностей по кредиту ниже чем у состоящих в браке: на 0.97% и 0.43% соответственно. Это может быть связано с недостаточно репрезентативной выборкой. 

### Вывод

Среди состоящих в браке задолженностей по кредитам меньше чем среди одиноких либо сожительствующих заемщиков, зависимость есть.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Построим сводную таблицу:

In [62]:
income_category_pivot = data.pivot_table(index=['income_category'], values='debt', aggfunc=['count','sum'])
income_category_pivot['%'] = income_category_pivot['sum'] / income_category_pivot['count'] * 100
income_category_pivot.sort_values('%')

,count,sum,%
,debt,debt,
income_category,,,
30-50,350,21,6.000000
>200,5067,358,7.065325
50-100,4091,331,8.090931
150-200,4764,405,8.501259
100-150,7160,624,8.715084
10-30,22,2,9.090909


В сводной таблице, отсортированной по возрастанию доли заемщиков по каждой категории дохода видно что между уровнем дохода и количеством задолженностей по кредиту зависмоисть не прослеживается.

### Вывод

Среди выделенных нами категория уровней доходов зависимость между доходом и возвратом кредита в срок не прослеживается.

- Как разные цели кредита влияют на его возврат в срок?

Построим сводную таблицу:

In [63]:
income_category_pivot = data.pivot_table(index=['real_purpose'], values='debt', aggfunc=['count','sum'])
income_category_pivot['%'] = income_category_pivot['sum'] / income_category_pivot['count'] * 100
income_category_pivot.sort_values('%')

,count,sum,%
,debt,debt,
real_purpose,,,
жилье,5690,397,6.977153
недвижимость,5121,385,7.518063
свадьба,2324,186,8.003442
образование,4013,370,9.220035
автомобиль,4306,403,9.359034


### Вывод

Проведенная на ранних стадиях лемматизация целей кредита позволила наглядно увидеть что кредиты взятые на приобретение жилья чаще всех остальных возвращаются без задолженностей, в то время как среди кредитов взятых на приобретение автомобиля внушительные 9,35% возвращаются с задержками.

Неожиданным стал тот факт, что высокая доля кредитов на образование так же не возвращается в срок.

### Шаг 4. Общий вывод

Заказчиком перед нами ставилась задача узнать влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. В результате исследования на оба утверждения можно ответить положительно:
1. Наличие детей увеличивает вероятность того что кредит не будет возвращен в срок, причем при снижении уровня дохода заемщика эта вероятность возрастает;
2. Заемщики состоящие в законном браке чаще возвращают кредит в срок чем одинокие или не узаконившие свои отношения.

Дополнительно была проверена гипотеза о том что между уровнем дохода и возвратом кредита в срок существует зависимость - гипотеза не подтвердилась.

Кроме того мы проверили как цель кредита влияет на его возврат в срок и вяснили что наиболее часто не возвращают в срок кредиты взятые на приобретение автомобиля и получения образования, а наиболее часто возвращаются в срок кредиты на приобретение недвижимости, как жилой так и не жилой.